In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore

In [80]:
leokas = ['ucr_leoka_monthly_1960_2020_dta/leoka_monthly_2017.dta',
          'ucr_leoka_monthly_1960_2020_dta/leoka_monthly_2016.dta',
          'ucr_leoka_monthly_1960_2020_dta/leoka_monthly_2015.dta',
          'ucr_leoka_monthly_1960_2020_dta/leoka_monthly_2014.dta',
          'ucr_leoka_monthly_1960_2020_dta/leoka_monthly_2013.dta']
leoka_17, leoka_16, leoka_15, leoka_14, leoka_13 = pd.read_stata(leokas[0]), pd.read_stata(leokas[1]), pd.read_stata(leokas[2]), pd.read_stata(leokas[3]), pd.read_stata(leokas[4])

dfs = [leoka_13, leoka_14, leoka_15, leoka_16, leoka_17]
dfs = [i.iloc[:,:-198] for i in dfs]

for j in dfs:
    print(j.shape)

(266424, 58)
(267972, 58)
(270288, 58)
(271740, 58)
(273408, 58)


In [81]:
df = pd.concat(dfs, ignore_index=True)
df.shape

(1349832, 58)

In [82]:
month_map = {'january': 1, 'february': 2, 'march': 3, 'april': 4, 'may': 5, 'june': 6, 'july': 7, 'august': 8, 'september': 9, 'october': 10, 'november': 11, 'december': 12}
df['month_code'] = df['month'].map(month_map) + (df['year'] - 2013)*12

In [83]:
#filter into assaults ever
df['total_assaults'] = df['assaults_no_injury_total'] + df['assaults_with_injury_total']
officer_assaulted = (df['total_assaults'] > 0)
ori_to_keep = df.loc[officer_assaulted, 'ori'].unique()
df = df[df['ori'].isin(ori_to_keep)]
df.shape #shape is 1286712 x 60 (about half the municipalities were retained)

(532356, 60)

In [84]:
#add sizes for all municipalities according to first observation per municipality
df['first_pop'] = df.groupby('ori')['population'].transform('first')
df['first_employment'] = df.groupby('ori')['total_employees_officers'].transform('first')

In [85]:
#column transformations

#drop 0 pop rows to avoid division by 0
df = df.loc[df['first_pop'] > 0]

#force size mechanism construction
median_condition = df['first_employment'] > df['first_employment'].median()
trueval, falseval = 1, 0
df['employment_median_indicator'] = np.where(median_condition, trueval, falseval)

#proportion mechanism
df['employment_pop_proportion'] = df['first_employment']/df['first_pop']
prop_median_condition = df['employment_pop_proportion'] > df['employment_pop_proportion'].median()
df['employment_pop_prop_indicator'] = np.where(prop_median_condition, trueval, falseval)

In [86]:
df = df.sort_values(by=['ori', 'month_code'])

In [87]:
for i in range(7):
    df[f'PreviousTreatment_{i}'] = df.groupby('ori')['officers_killed_total'].shift(i)
    df[f'post_{i}'] = df[f'PreviousTreatment_{i}'] > 0

    #don't do the idxmin thing, shouldn't matter since things are NaN where edge case shifts; drop the NaNs by removing the PreviousTreament_i columns

    df[f'post_{i}'] = df[f'post_{i}'].astype(int)
    df = df.drop(columns=[f'PreviousTreatment_{i}'])

In [88]:
#make three pre-treatment month indicators
for i in range(1, 4, 1):
    df[f'FutureTreatment_{i}'] = df.groupby('ori')['officers_killed_total'].shift(-1*i)
    df[f'pre_{i}'] = df[f'FutureTreatment_{i}'] > 0
    df[f'pre_{i}'] = df[f'pre_{i}'].astype(int)
    df = df.drop(columns=[f'FutureTreatment_{i}'])

In [89]:
df['assault_indicator'] = (df['total_assaults'] > 0).astype(int)

In [90]:
ucrs = ['ucr_arrests_monthly_index_1974_2018_dta/ucr_arrests_monthly_index_crimes_age_2017.dta',
        'ucr_arrests_monthly_index_1974_2018_dta/ucr_arrests_monthly_index_crimes_age_2016.dta',
        'ucr_arrests_monthly_index_1974_2018_dta/ucr_arrests_monthly_index_crimes_age_2015.dta',
        'ucr_arrests_monthly_index_1974_2018_dta/ucr_arrests_monthly_index_crimes_age_2014.dta',
        'ucr_arrests_monthly_index_1974_2018_dta/ucr_arrests_monthly_index_crimes_age_2013.dta']

ucr_17, ucr_16, ucr_15, ucr_14, ucr_13 = pd.read_stata(ucrs[0]), pd.read_stata(ucrs[1]), pd.read_stata(ucrs[2]), pd.read_stata(ucrs[3]), pd.read_stata(ucrs[4])


In [91]:
ucr_dfs = [ucr_13, ucr_14, ucr_15, ucr_16, ucr_17]
ucr = pd.concat(ucr_dfs, ignore_index=True)
ucr['total_arrests'] = ucr['theft_tot_arrests'] + ucr['robbery_tot_arrests'] + ucr['rape_tot_arrests'] + ucr['murder_tot_arrests'] + ucr['mtr_veh_theft_tot_arrests'] + ucr['burglary_tot_arrests'] + ucr['arson_tot_arrests'] + ucr['agg_assault_tot_arrests']
ucr['log_total_arrests'] = np.log(ucr['total_arrests'] + 1)

/var/folders/5h/mg9jv4jj4_zc1dsmj4zsf1vm0000gn/T/ipykernel_65305/2892541529.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ucr['total_arrests'] = ucr['theft_tot_arrests'] + ucr['robbery_tot_arrests'] + ucr['rape_tot_arrests'] + ucr['murder_tot_arrests'] + ucr['mtr_veh_theft_tot_arrests'] + ucr['burglary_tot_arrests'] + ucr['arson_tot_arrests'] + ucr['agg_assault_tot_arrests']
/Users/srinidhi/Library/Python/3.9/lib/python/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/var/folders/5h/mg9jv4jj4_zc1dsmj4zsf1vm0000gn/T/ipykernel_65305/2892541529.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` man

In [92]:
ucr['month_code'] = ucr['month'].map(month_map) + (ucr['year'] - 2013)*12
df = df.drop_duplicates(subset=['ori', 'month_code'])
ucr = ucr.drop_duplicates(subset=['ori', 'month_code'])

/var/folders/5h/mg9jv4jj4_zc1dsmj4zsf1vm0000gn/T/ipykernel_65305/2433599158.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ucr['month_code'] = ucr['month'].map(month_map) + (ucr['year'] - 2013)*12


In [93]:
ucr.shape
merged = pd.merge(df, ucr, on=['ori', 'month_code', 'year'], how='left')
df.shape, ucr.shape, merged.shape

((465876, 76), (846711, 632), (465876, 705))

In [94]:
#data to run first two stage regressions
merged.to_stata('event_study.dta')

In [95]:
#dataset creation for stage 3
overall_conditions = [(merged['post_0'] + merged['post_1'] + merged['post_2'] + merged['post_4'] >= 1),
                      (merged['pre_1'] + merged['pre_2'] + merged['pre_3'] >= 1)]
overall_condition_values = [1, 0] #2 is default value
merged['post_overall'] = np.select(overall_conditions, overall_condition_values, default=2)

In [96]:
avg_data = merged.loc[merged['post_overall'] < 2]

In [97]:
merged.shape, avg_data.shape

((465876, 706), (1467, 706))

In [98]:
avg_data.to_stata('DDD.dta')

In [99]:
for i in merged.columns:
    print(i)

ori
agency_name_x
state_x
state_abb_x
number_of_months_reported_x
year
month_x
date
ori9_x
fips_state_code_x
fips_county_code_x
fips_state_county_code_x
fips_place_code_x
agency_type
crosswalk_agency_name
census_name
longitude
latitude
address_name
address_street_line_1
address_street_line_2
address_city
address_state
address_zip_code
population_x
population_group_x
country_division_x
msa
report_indicator
record_indicator
month_indicator
covered_by_x
shift_data
no_male_female_breakdown
assault_injury_indicator
assault_no_injury_indicator
male_employees_officers
female_employees_officers
total_employees_officers
male_employees_civilians
female_employees_civilians
total_employees_civilians
male_employees_total
female_employees_total
total_employees_total
officers_killed_total
officers_killed_by_felony
officers_killed_by_accident
assaults_with_injury_gun
assaults_with_injury_knife
assaults_with_injury_oth_weap
assaults_with_injury_unarmed
assaults_with_injury_total
assaults_no_injury_gun
